In [6]:
from collections import OrderedDict
from pymatgen import MPRester
from pymatgen.io.ase import AseAtomsAdaptor
import requests
import json

In [7]:
material_ids = OrderedDict([
    ('diamond', 'mp-66'),  # C
    ('gaas', 'mp-2534'),  # GaAs
    ('rocksalt', 'mp-22862'),  # NaCl
    ('cubic_perovskite', 'mp-2998'),  # BaTiO3
    ('tetragonal_perovskite', 'mp-5986'),  # BaTiO3
    ('trigonal_perovskite', 'mp-5020'),  # BaTiO3
    ('orthorhombic_perovskite', 'mp-5777'),  # BaTiO3
    ('zincblende', 'mp-10695'),  # ZnS
    ('wurtzite', 'mp-10281'),  # ZnS
    ('fcc', 'mp-23'),  # Ni
    ('big_fcc', 'mp-76'),  # Sr
    ('bcc', 'mp-13'),  # Fe
    ('big_bcc', 'mp-70'),  # Rb
    ('hcp', 'mp-153'),  # Mg
    ('trigonal', 'mp-782'),  # Te2Pd
    ('tetragonal', 'mp-742'),  # Ti2Cu
    ('monoclinic', 'mp-684'),  # BaS2
    ('triclinic', 'mp-9122'),  # CaP3
    ('orthorhombic', 'mp-872')  # BaSn
])

In [8]:
with MPRester('0WqdPfXxloze6T9N') as mpr:
    structures = OrderedDict([(name, mpr.get_structure_by_material_id(id)) for name, id in material_ids.items()])

### soap
---

In [9]:
def get_spkit(structure):
    '''
    Get the spkit dictionary from a pymatgen Structure object
    '''
    spkit = {}
    for specie in structure.species:
        if specie.Z in spkit.keys():
            spkit[specie.Z] += 1
        else:
            spkit[specie.Z] = 1
    return spkit

In [10]:
def get_spkitMax(structures):
    '''
    Get the spkitMax dictionary from a list of pymatgen Structure objects
    '''
    spkitMax = {}
    spkits = [get_spkit(structure) for structure in structures]
    for spkit in spkits:
        for Z in spkit.keys():
            if Z in spkitMax.keys():
                if spkitMax[Z] < spkit[Z]:
                    spkitMax[Z] = spkit[Z]
            else:
                spkitMax[Z] = spkit[Z]
    return spkitMax

In [ ]:
soaps = get_Soaps(qps.values(),
                  nocenters=None, chem_channels=False,
                  centerweight=1.0, gaussian_width=0.5,
                  cutoff=2.0, cutoff_transition_width=0.5,
                  nmax=16, lmax=14,
                  spkitMax=None, nprocess=8,
                  chemicalProjection=None, dispbar=False,
                  is_fast_average=False)

In [28]:
%%time
endpoint = "fingerprints"
function = "soaps"
payload = {
    'structures': [json.dumps(s.as_dict()) for s in structures.values()],
}

r = requests.get('http://127.0.0.1:9090/v1/{}/{}'.format(endpoint, function), params=payload)

CPU times: user 384 ms, sys: 136 ms, total: 520 ms
Wall time: 43 s


### Matminer
---

In [30]:
endpoint = "fingerprints"
function = "matminer"
payload = {
    'structures': json.dumps([s.as_dict() for s in structures.values()]),
    'preset': 'cn',
    'crystal_site_args': {},
    'site_stats_args': {}
}

r = requests.get('http://127.0.0.1:9090/v1/{}/{}'.format(endpoint, function), params=payload)
print(r.json())

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.306986234815436, 0.25065648775005495, 0.0, 0.0, 0.0024952054781244275, 0.0027537491661462866, 0.0, 0.0, 0.4905579918336536, 0.31615856111875873, 0.0, 0.0, 0.0003574319443757415, 0.000714863888751483, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.19960313592841022, 0.3992062

In [31]:
endpoint = "distances"
function = "matminer"
payload = {
    'structures': json.dumps([s.as_dict() for s in structures.values()]),
    'preset': 'cn',
    'crystal_site_args': {},
    'site_stats_args': {}
}

r = requests.get('http://127.0.0.1:9090/v1/{}/{}'.format(endpoint, function), params=payload)
print(r.json())

[0.0, 1.4142135623730951, 1.3007319857360136, 1.1753141221924452, 1.1974160592254943, 1.1366017882503427, 0.0, 0.00832729830992438, 1.4142135623730951, 1.4142135623730951, 1.2298537750101761, 1.4142135623730951, 1.4100584278389627, 1.294727986554352, 0.895992975199486, 1.292927830326727, 1.059553768321252, 1.2150181851488095, 1.4142135623730951, 1.3007319857360136, 1.1753141221924452, 1.1974160592254943, 1.1366017882503427, 0.0, 0.00832729830992438, 1.4142135623730951, 1.4142135623730951, 1.2298537750101761, 1.4142135623730951, 1.4100584278389627, 1.294727986554352, 0.895992975199486, 1.292927830326727, 1.059553768321252, 1.2150181851488095, 0.8460367167006965, 0.8665582026815353, 0.8436436738920159, 0.8337702651201839, 1.4142135623730951, 1.4112848243802567, 1.4142135623730951, 1.4142135623730951, 1.2298537750101761, 1.4142135623730951, 1.405866231870177, 0.7339347457938654, 1.1833590825306934, 1.3173812462694114, 1.0076881392059924, 1.2150181851488095, 0.315952307754511, 0.3123971625

In [32]:
endpoint = "comparisons"
function = "matminer"
payload = {
    'structures': json.dumps([s.as_dict() for s in structures.values()]),
    'preset': 'cn',
    'crystal_site_args': {},
    'site_stats_args': {}
}

r = requests.get('http://127.0.0.1:9090/v1/{}/{}'.format(endpoint, function), params=payload)
print(r.json())

[[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0

### pymatgen
---

In [33]:
endpoint = "distances"
function = "pymatgen"
payload = {
        'structures': json.dumps([s.as_dict() for s in structures.values()]),
        'comparator': 'OccupancyComparator',
    }

r = requests.get('http://127.0.0.1:9090/v1/{}/{}'.format(endpoint, function), params=payload)
print(r.json())

[[0.0, 1.6967990814344004e-16], None, None, None, None, None, [0.0, 0.0], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [0.0, 1.0447481101469657e-16], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [0.04162903049864667, 0.06282144522146531], [0.04484153660633773, 0.07170507568892255], [0.04406201068854451, 0.06950299279838568], None, None, None, None, None, None, None, None, None, None, None, None, [0.03941790205195098, 0.06150971374993145], [0.032750811047071816, 0.050367877212565956], None, None, None, None, None, None, None, None, None, None, None, None, [0.026880600961174998, 0.04216592208900977], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None

In [17]:
# TODO: Fix struct_1 and struct_2 references
endpoint = "comparisons"
function = "pymatgen"
payload = {
        'structures': json.dumps([s.as_dict() for s in structures.values()]),
        'comparator': 'OccupancyComparator',
    }

r = requests.get('http://127.0.0.1:5000/v1/{}/{}'.format(endpoint, function), params=payload)
print(r.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### stidy
---

In [21]:
endpoint = "fingerprints"
function = "stidy"
payload = {
        'structures': json.dumps([s.as_dict() for s in structures.values()]),
        'symprec': 1e-2,
        'angle_tolerance': 5.
    }
r = requests.get('http://127.0.0.1:5000/v1/{}/{}'.format(endpoint, function), params=payload)
print(r.json())

['227_01Aa', '216_01Ba_01Ac', '225_01Ba_01Ab', '221_01Ca_01Bb_01Ah', '123_01Ca_01Ab_01Bd_01Ai', '166_01Ca_01Bb_01Ah', '65_01Ca_01Bc_01Ad_01Ap', '216_01Ba_01Ac', '186_01A01Ba_01A01Bb', '225_01Aa', '225_01Aa', '229_01Aa', '229_01Aa', '194_01Ac', '164_01Ba_01Ad', '139_01Ba_01Ae', '15_01Be_01Af', '2_01B03Ai', '63_01B01Ac']


In [ ]:
endpoints = {# 'distances': ['matminer', 'pymatgen', 'soaps'],
             # 'comparisons': ['matminer', 'pymatgen', 'soaps'],
             # 'fingerprints': ['matminer', 'stidy', 'soaps'],
             'fingerprints': ['soaps']}
for endpoint, functions in endpoints.items():
    for function in functions:
        print(endpoint, function)
        payload = payloads[function]
        r = requests.get('http://127.0.0.1:5000/v1/{}/{}'.format(endpoint, function), params=payload)
        results = json.loads(r.text)
        print(results)